### A Park for every neighbourhood in Toronto
by _Marc Behrens_

In [1]:
# import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
# Import the dataframe of Postalcodes, Boroughs and neighbourhoods of Toronto created in the notebook submitted before.
# The Toronto neighbourhoods with its coordinates is a dataframe we already have created in earlier exercises. They were created on basis of:
# + the Wikipedia-Page 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# + a csv-file with the coordinates of the postal codes in Toronto: https://cocl.us/Geospatial_data
df_toronto = pd.read_csv("toronto-neighbourhoods-coord.csv")

In [3]:
df_toronto.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [4]:
# map of Toronto with the neighborhoods.
map_toronto = folium.Map(location=[43.679563, -79.377529], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [5]:
# Foursquare credentials
CLIENT_ID = '1EX5D3DJHD5WVXUQ5JOFATMJ3S5BNRYBETZVAG1RDPH00YLC' # your Foursquare ID
CLIENT_SECRET = 'UNR20VCVBPYVVLQVCKJM443FK1LP4FIVWLJNYU4TTW3QFLUV' # your Foursquare Secret
VERSION = '20201108'#'20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Your credentails:
CLIENT_ID: 1EX5D3DJHD5WVXUQ5JOFATMJ3S5BNRYBETZVAG1RDPH00YLC
CLIENT_SECRET:UNR20VCVBPYVVLQVCKJM443FK1LP4FIVWLJNYU4TTW3QFLUV


In [6]:
# Funtion to get the nearby parks of the neighbourhoods
def getNearbyParks(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()["response"]['groups'][0]['items']
        
        response = requests.get(url).json()["response"]
        
        if "groups" in response and "items" in response['groups'][0] :
            for item in response['groups'][0]['items'] :
                if item['venue']['categories'][0]['name'] == 'Park' :
                    venues_list.append((
                        name, 
                        lat, 
                        lng, 
                        item['venue']['name'], 
                        item['venue']['location']['lat'], 
                        item['venue']['location']['lng'],  
                        item['venue']['categories'][0]['name']))

    nearby_parks = pd.DataFrame(venues_list, columns=['Neighborhood', 
                                                        'Neighborhood Latitude', 
                                                        'Neighborhood Longitude', 
                                                        'Venue', 
                                                        'Venue Latitude', 
                                                        'Venue Longitude', 
                                                        'Venue Category'])
    
    
    return(nearby_parks)

In [7]:
toronto_parks = getNearbyParks(names=df_toronto['Neighbourhood'], latitudes=df_toronto['Latitude'], longitudes=df_toronto['Longitude'])
toronto_parks.shape

(57, 7)

In [8]:
# Count Parks by neighborhood
toronto_only_parks=toronto_parks[['Neighborhood', 'Venue Category']].groupby(['Neighborhood']).count()
toronto_only_parks

,Venue Category
Neighborhood,
"Bathurst Manor, Wilson Heights, Downsview North",1
Berczy Park,1
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",1
Caledonia-Fairbanks,2
Central Bay Street,1
Christie,2
Church and Wellesley,1
"Commerce Court, Victoria Hotel",1
Davisville,1


In [9]:
# Rename column "Vanue Category" to "Parks"
toronto_only_parks.rename(columns={'Venue Category': 'Parks'},inplace=True)

In [10]:
# Preparing df_toronto to join the toronto_only_parks
df_toronto.head(2)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572


In [11]:
# Set Neighbourhood as index and drop unnamed olumn
df_toronto.set_index('Neighbourhood', inplace=True)
df_toronto.drop("Unnamed: 0", axis=1, inplace=True)
df_toronto.head(2)

,Postal Code,Borough,Latitude,Longitude
Neighbourhood,,,,
Parkwoods,M3A,North York,43.753259,-79.329656
Victoria Village,M4A,North York,43.725882,-79.315572


In [12]:
# Join the dataframes df_toronto and toronto_parks in order to add the number of parks to the rest.
toronto_parks=df_toronto.join(toronto_only_parks)
toronto_parks.head(10)

,Postal Code,Borough,Latitude,Longitude,Parks
Agincourt,M1S,Scarborough,43.794200,-79.262029,NaN
"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484,NaN
"Bathurst Manor, Wilson Heights, Downsview North",M3H,North York,43.754328,-79.442259,1.0
Bayview Village,M2K,North York,43.786947,-79.385975,NaN
"Bedford Park, Lawrence Manor East",M5M,North York,43.733283,-79.419750,NaN
Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306,1.0
"Birch Cliff, Cliffside West",M1N,Scarborough,43.692657,-79.264848,NaN
"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto,43.636847,-79.428191,NaN
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",M7Y,East Toronto,43.662744,-79.321558,1.0
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",M5V,Downtown Toronto,43.628947,-79.394420,NaN


In [13]:
# Change NaN to 0 and the column to int
toronto_parks['Parks']=toronto_parks['Parks'].fillna(0).astype(int)
toronto_parks.head(10)

,Postal Code,Borough,Latitude,Longitude,Parks
Agincourt,M1S,Scarborough,43.794200,-79.262029,0
"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484,0
"Bathurst Manor, Wilson Heights, Downsview North",M3H,North York,43.754328,-79.442259,1
Bayview Village,M2K,North York,43.786947,-79.385975,0
"Bedford Park, Lawrence Manor East",M5M,North York,43.733283,-79.419750,0
Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306,1
"Birch Cliff, Cliffside West",M1N,Scarborough,43.692657,-79.264848,0
"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto,43.636847,-79.428191,0
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",M7Y,East Toronto,43.662744,-79.321558,1
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",M5V,Downtown Toronto,43.628947,-79.394420,0


In [14]:
toronto_parks.head(2)

,Postal Code,Borough,Latitude,Longitude,Parks
Agincourt,M1S,Scarborough,43.794200,-79.262029,0
"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484,0


In [15]:
# How many neighbourhoods do not have parks nearby?
toronto_parks['Parks'].value_counts()

0    56
1    32
2    14
3     1
Name: Parks, dtype: int64

In [16]:
# Map of Neighbourhoods with parks
map_toronto_parks = folium.Map(location=[43.679563, -79.377529], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, parks in zip(toronto_parks['Latitude'], toronto_parks['Longitude'], toronto_parks['Borough'], toronto_parks.index, toronto_parks['Parks']):
    label = '{}: {}'.format(neighborhood, parks)
    label = folium.Popup(label, parse_html=True)
    parkIndicatorColor='green'
    
    
    if (parks == 0) :
        parkIndicatorColor='red'
    elif (parks == 1) :
        parkIndicatorColor='orange'
    elif (parks == 2) :
        parkIndicatorColor='lightgreen'
    else :
        parkIndicatorColor='green'
    
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=parkIndicatorColor,
        fill=True,
        fill_color='grey',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_parks)  
    
map_toronto_parks

In [17]:
# Get dataframe of neighbourhoods without parks.
toronto_no_parks = toronto_parks[toronto_parks.Parks == 0]
toronto_no_parks.head()

,Postal Code,Borough,Latitude,Longitude,Parks
Agincourt,M1S,Scarborough,43.794200,-79.262029,0
"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484,0
Bayview Village,M2K,North York,43.786947,-79.385975,0
"Bedford Park, Lawrence Manor East",M5M,North York,43.733283,-79.419750,0
"Birch Cliff, Cliffside West",M1N,Scarborough,43.692657,-79.264848,0


We have seen before that there are 56 neighbourhoods without park. we cannot build a park in all that locations immediately, but we will start with 5 neighbourhoods.

In order to not have them all in the same side of the city, we will create 5 clusters.

In [18]:
# Set up matrix to use K-Means
X=toronto_no_parks[['Latitude', 'Longitude']]
print(type(X))

<class 'pandas.core.frame.DataFrame'>


In [19]:
# Create a k-Means instance for 5 clusters
k_means =  KMeans(init="k-means++", n_clusters=5)

In [20]:
# Execute the algorithm on the coordinates of the neighbourhoods without parks.
k_means.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [21]:
k_means.labels_

array([4, 1, 3, 3, 0, 2, 2, 1, 4, 0, 0, 1, 3, 3, 0, 1, 3, 2, 4, 3, 1, 1,
       1, 0, 1, 3, 4, 1, 2, 1, 1, 1, 2, 0, 2, 4, 1, 1, 0, 1, 3, 2, 0, 2,
       1, 2, 2, 4, 0, 1, 1, 0, 3, 3, 4, 3], dtype=int32)

In [22]:
# We create a data frame with the toronto_no_parks-infomation plus the cluster.
# This will give us a warning, but we know what we are doing as the labels-array is the result of the coordinates of the toronto_no_parks-data frame.
df_no_parks_clustered = toronto_no_parks
df_no_parks_clustered['cluster'] = k_means.labels_

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
df_no_parks_clustered.head(6)

,Postal Code,Borough,Latitude,Longitude,Parks,cluster
Agincourt,M1S,Scarborough,43.794200,-79.262029,0,4
"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484,0,1
Bayview Village,M2K,North York,43.786947,-79.385975,0,3
"Bedford Park, Lawrence Manor East",M5M,North York,43.733283,-79.419750,0,3
"Birch Cliff, Cliffside West",M1N,Scarborough,43.692657,-79.264848,0,0
"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto,43.636847,-79.428191,0,2


In [24]:
# Map of Neighbourhoods with parks
map_toronto_no_parks_clustered = folium.Map(location=[43.679563, -79.377529], zoom_start=11)

clusterColors = ['red', 'orange', 'yellow', 'purple', 'green']

# add markers to map
for lat, lng, neighborhood, cluster in zip(df_no_parks_clustered['Latitude'], df_no_parks_clustered['Longitude']
                                                    , df_no_parks_clustered.index, df_no_parks_clustered['cluster']):
    label = '{} ({} cluster)'.format(neighborhood, clusterColors[cluster])
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=clusterColors[cluster],
        fill=True,
        fill_color='grey',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_no_parks_clustered)  

    
# Add cluster-centers
for i in range(5) :
    label = 'Center of {} cluster'.format(clusterColors[i])
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        k_means.cluster_centers_[i],  #[lat, lng],
        radius=10,
        popup=label,
        color=clusterColors[i],
        fill=True,
        fill_color=clusterColors[i],
        fill_opacity=1,
        parse_html=False).add_to(map_toronto_no_parks_clustered)  
    
    
map_toronto_no_parks_clustered

So, now we have five clusters from where to chose the neighbourhoods where new parks are to be created.
Our choices are:
<li> Brockton, Parkdale Village, Exhibition Place (green cluster)
<li> Alderwood, Long Branch (yellow cluster)
<li> Bedford Park, Lawrence Manor East (red cluster)
<li> The Beaches (purple cluster)
<li> Wexford, Maryvale (purple cluster)